In [ ]:
import os
import pickle
#import tomotopy as tp
import pandas as pd
from dateutil import parser
from langdetect import detect, DetectorFactory
import ast

In [ ]:
import runtimex
import importlib
importlib.reload(runtimex)

In [ ]:
article_list = []
with open("poynter_crawl_overall_list.pkl", "rb") as f:
    article_list = pickle.load(f)
with open("poynter_crawl_overall_list_2.pkl", "rb") as f:
    article_list_2 = pickle.load(f)
article_list.extend(article_list_2)
print("There are " + str(len(article_list)) + " articles in the database")


In [ ]:
#We examine the publish date of the articles, removing those from after March. 25th, 2020
new_article_list = []
for each_article in article_list:
    publish_date_str = each_article[1]
    #print(publish_date)
    try:
        publish_date = parser.parse(publish_date_str)
        if publish_date.replace(tzinfo=None) < parser.parse("2020-03-26 00:00:00+00:00").replace(tzinfo=None):
            new_article_list.append(each_article)
    except Exception as e:
        #If there is no date we have to include it.
        #print(e)
        new_article_list.append(each_article)
print("There are now " + str(len(new_article_list)) + " articles published before 2020-03-26.")

In [ ]:
DetectorFactory.seed = 0
english_list = []
for each_article in new_article_list:
    if each_article != '':
        try:
            if detect(each_article[4][0]) == 'en':
                english_list.append(each_article)
        except Exception as e:
            pass

print("There are " + str(len(english_list)) + " english articles in database.")

In [ ]:
os.environ["TOKENIZERS_PARALLELISM"] = "true"
import torch.multiprocessing as multiprocessing
if multiprocessing.get_start_method() == 'fork':
    multiprocessing.set_start_method('spawn', force=True)
process_list = []
device_num = 0
corpus_main = "/"
for each_keyword in os.listdir(corpus_main):
    p = multiprocessing.Process(target=runtimex.Extraction_comparison_of_keyword, args=(each_keyword, device_num, english_list))
    device_num += 1
    if device_num >= 8:
        device_num = 0
    process_list.append(p)

for p in process_list:
    p.start()
for p in process_list:
    p.join()

In [ ]:
os.environ["TOKENIZERS_PARALLELISM"] = "true"
import torch.multiprocessing as multiprocessing
if multiprocessing.get_start_method() == 'fork':
    multiprocessing.set_start_method('spawn', force=True)
process_list = []
device_num = 0
corpus_main = "/"
for each_keyword in os.listdir(corpus_main):
    p = multiprocessing.Process(target=runtimex.Dup_removal, args=(each_keyword,))
    device_num += 1
    if device_num >= 8:
        device_num = 0
    process_list.append(p)

for p in process_list:
    p.start()
for p in process_list:
    p.join()

In [ ]:
os.environ["TOKENIZERS_PARALLELISM"] = "true"
import torch.multiprocessing as multiprocessing
if multiprocessing.get_start_method() == 'fork':
    multiprocessing.set_start_method('spawn', force=True)
process_list = []
device_num = 0
corpus_main = "/"
for each_keyword in os.listdir(corpus_main):
    p = multiprocessing.Process(target=runtimex.Comparison_analysis, args=(each_keyword,))
    device_num += 1
    if device_num >= 8:
        device_num = 0
    process_list.append(p)

for p in process_list:
    p.start()
for p in process_list:
    p.join()


In [ ]:
overall_similarity = list(df["6"])
thresholds = [0.65, 0.6, 0.55, 0.5, 0.45, 0.4, 0.35, 0.3]
for threshold in thresholds:
    count = sum(1 for number in overall_similarity if number > threshold)
    print(f'Numbers greater than {threshold}: {count}')

In [ ]:
stat_dir = "/"
combined_df = pd.DataFrame()
for keyword in os.listdir(stat_dir):
    keyword_dir = os.path.join(stat_dir, keyword)
    file_name = keyword_dir + "/output_allthreshold.csv"
    current_df = pd.read_csv(file_name)
    combined_df = pd.concat([combined_df, current_df], ignore_index=True)

In [ ]:
combined_df

In [ ]:
combined_df.to_csv("Comparison_Combined_Dataframe_all_threshold.csv")